## Import libraries and load files

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import SCORERS

from sklearn.model_selection import KFold
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from warnings import warn

import numpy as np
import pandas as pd

In [0]:
#Read the data using the Unnamed (probably id) as index
url = 'https://s3.amazonaws.com/drivendata/data/4/public/81e8f2de-9915-4934-b9ae-9705685c9d50.csv'
#url = '../src/data/raw/training.csv'
training = pd.read_csv(url, index_col='Unnamed: 0')

labels = ['Function', 'Object_Type', 'Operating_Status', 'Position_Type', 'Pre_K', 'Reporting', 
          'Sharing', 'Student_Type', 'Use']

numeric = ['FTE', 'Total']

categoric = [ 'Facility_or_Department', 'Function_Description', 
            'Fund_Description', 'Job_Title_Description', 'Location_Description', 
            'Object_Description', 'Position_Extra', 'Program_Description', 'SubFund_Description', 
            'Sub_Object_Description', 
            'Text_1', 'Text_2', 'Text_3', 'Text_4']

### FunctionTransformers

In [0]:
# Define combine_text_columns()
def combine_text_columns(data_frame):
    """ converts all text in each row of data_frame to single vector """
    
    # Drop non-text columns that are in the df
    text_data = data_frame[categoric]
    
    # Replace nans with blanks
    text_data.fillna("", inplace=True)
    
    for category in categoric:
      training.loc[:,category] = training[category].str.lower()
    
    
    # Join all text items in a row that have a space in between
    return text_data.apply(lambda x: " ".join(x), axis=1)

In [0]:
groupped_FTE = training[['FTE', 'Object_Type']].groupby(by='Object_Type')
groupped_total = training[['Total', 'Object_Type']].groupby(by='Object_Type')
# Define combine_numeric_columns()
def combine_numeric_columns(data_frame, groupped_FTE=groupped_FTE, groupped_total=groupped_total):
    """ process all the numeric data """
    
    # Drop non-numeric columns that are in the df
    data = data_frame[numeric]
    
    #Remove inconsistent data
    data.loc[(data[numeric[0]] < 0) | (data[numeric[0]] > 1), numeric[0]] = np.nan
    data.loc[(data[numeric[1]] > 1) | (data[numeric[1]] < 0), numeric[1]] = np.nan
    
    #Impute the missing data with the median from each class
    for group in groupped_FTE.median().index:
      indexes_FTE = groupped_FTE.get_group(group).index.values
      indexes_total = groupped_total.get_group(group).index.values
      data.loc[ data.FTE.isnull() & np.isin(data.index.values,indexes_FTE), 'FTE'] = groupped_FTE.median().loc[group, "FTE"]
      data.loc[ data.Total.isnull() & np.isin(data.index.values,indexes_total), 'Total'] = groupped_total.median().loc[group,"Total"]
      
    return data

In [0]:
# Preprocess the text data: get_text_data
get_text_data = FunctionTransformer(combine_text_columns, validate=False)

# Preprocess the numeric data: get_numeric_data
get_numeric_data = FunctionTransformer(combine_numeric_columns, validate=False)



In [0]:
# Recover the targets and split the data
y = pd.get_dummies(training['Object_Type'])
X = training.drop(columns=labels)

### Pipeline

Apply the transformations on numeric and categorica data. Neither dimension reduction or standard scaler are used.

In [0]:
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imp', SimpleImputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer',HashingVectorizer(token_pattern="[A-Za-z0-9]+(?=\\s+)", 
                                                    norm=None, 
                                                    binary=False,
                                                    ngram_range=(1,2)) 
                    )
                ]))
             ]
        )),
        ('reduce_dim', TruncatedSVD(n_components = 50)),
        ('clf', DecisionTreeClassifier())
        
    ])

Applying the steps, we got a sparse matrix with 1048578 features.

## Training
The model is trained and tested using k-fold with k = 10. The GridSearchCV.

In [0]:
# create a dictionary of all values we want to test for n_neighbors
# Test k = 3, k = 5, k = 7
param_grid = {'clf__max_depth': np.array([25, 10])}

# Metrics: acc, average_precision
metricas = [  'accuracy',
 'average_precision',
# 'balanced_accuracy', #error: multi-label indicator not supported# 'precision',
# 'precision_samples',
# 'recall',
 'roc_auc',
           ]

#use gridsearch to test all values for n_neighbors
knn_gscv = GridSearchCV(pl, param_grid=param_grid,
                    scoring=metricas,
                    verbose=1,
                    refit="accuracy",
                    return_train_score=False, cv=10)

#fit model to data
knn_gscv.fit(X, y)

Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#in

In [0]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

In [0]:
results = pd.DataFrame(knn_gscv.cv_results_)

In [0]:
results.head()

In [0]:
results.to_csv('dt_results.csv')

In [0]:
results.columns

In [0]:
results[results.columns[results.columns.str.endswith('test_accuracy')]]